In [1]:
# 막혔을때 예외처리
# 스킬 추가

In [1]:
# conda install beautifulsoup4
# conda install selenium
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
#import requests
from selenium.webdriver.common.by import By
import pandas as pd
import random
import re
import numpy as np
import sys
import copy

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        ## Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--blink-setting=imagesEnabled=false') # 이미지 로딩 안함
#options.add_argument('--mute-audio') # 음소거
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-site-isolation-trials")
#options.add_argument('--disable-setuid-sandbox')
options.add_argument('--incognito') # 시크릿모드
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36') # user와 같은 설정 주기
chromedriver = '../data/chromedriver'
try:
    driver.quit()
except NameError:
    pass
finally:
    driver = webdriver.Chrome(chromedriver, options=options) # 크롬 드라이버 설정

In [4]:
def remove_tags(string):
    return re.sub('<.+?>', '', str(string), 0).strip()

def save_1row(data):
    global Data
    length = Data.shape[0]
    Data.loc[length] = data
    return

def reset_Data():
    global Data
    Data = pd.DataFrame({}, columns=[\
        'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
        'rec type','url', 'keyword'])
    return

def get_job_keyword(job):
    global ijob, ikeyword
    job_list = ['데이터 엔지니어', '데이터 분석가', '데이터 사이언티스트', '머신러닝 엔지니어', \
                '데이터 플랫폼 개발자', 'DBA', '임베디드', 'IoT', '마켓팅 데이터', '정보보안', \
                '백엔드', '풀스택']
    job_keywords = {'데이터 엔지니어':['데이터 엔지니어'], \
                    '데이터 분석가': ['데이터 분석가'], \
                    '데이터 사이언티스트': ['데이터 사이언티스트'],\
                    '머신러닝 엔지니어': ['머신러닝 엔지니어', 'ML'], \
                    '데이터 플랫폼 개발자':['데이터 플랫폼 개발자'], \
                    'DBA' : ['DBA', '데이터베이스 관리자'], \
                    '임베디드':['임베디드'],\
                    'IoT':['IoT'],\
                    '마켓팅 데이터':['마켓팅 데이터'],\
                    '정보보안':['정보보안'],\
                    '백엔드':['백엔드', 'backend'],\
                    '풀스택':['풀스택', 'fullstack']}
    if job is '':
        ijob = 0
        ikeyword = 0
    elif ikeyword+1 == len(job_keywords[job]):
        ijob += 1
        ikeyword = 0
    else:
        ikeyword += 1
    if ijob == len(job_list):
        return ('None')                     ################# None 반환시 끝내기 
    job = job_list[ijob];
    keyword = job_keywords[job][ikeyword]
    return (job, keyword)

def get_job_html(base_url, job, keyword):
    driver.get(base_url)
    search_job = driver.find_element('xpath', '//*[@id="stext"]')
    search_job.send_keys(keyword+Keys.ENTER)
    time.sleep(3+3*random.random()) # 랜덤으로
    html = driver.page_source
    html_jobs = BeautifulSoup(html, 'html.parser')
    return (html_jobs)

def get_recruit_html(base_url, recruit):
    rec_url = recruit['href']
    search_url = base_url + rec_url
    driver.get(search_url)
    stime = 1+2*random.random()
    time.sleep(stime)
    html_recruit = BeautifulSoup(driver.page_source, 'html.parser')
    return (html_recruit, search_url)

def get_iframe_html(base_url, iframe):
    driver.switch_to.frame('gib_frame') # iframe으로 옮겨가기
    stime = 1+1*random.random()
    time.sleep(stime)
    html_iframe = BeautifulSoup(driver.page_source, 'html.parser')
    return (html_iframe)

def get_next_job_html(base_url, page_url):
    driver.get(base_url + page_url)
    stime = 3+2*random.random()
    html = driver.page_source
    html_jobs = BeautifulSoup(html, 'html.parser')
    return (html_jobs)

In [5]:
job = ''; keyword = ''
site = 'https://www.jobkorea.co.kr/'
reset_Data()

In [6]:
job, keyword = get_job_keyword(job)
html_jobs_1page = get_job_html(site, job, keyword)

In [ ]:
pages = html_jobs_1page.find_all('div', {'class':'tplPagination newVer wide'})[-1].select('li')
max_pages = len(pages)
print (f'\n\njob {job}')
print (f'keyword {keyword}')
print (f'ijob, ikeyword {ijob}, {ikeyword}')
print (f'max page {max_pages}')

max_pages = min(max_pages, 5)
ipage = 2
while ipage < max_pages:
    if ipage > 0:
        html_jobs = get_next_job_html(site, pages[ipage].select('a')[0]['href'])
    else:
        html_jobs = copy.copy(html_jobs_1page)
    recruit_default = html_jobs.find('div', {'class':'list-default'})
    recruit_items = recruit_default.find_all('li', {'class':'list-post'}) # list-post / list-post active

    print (f'\npage {ipage+1} / {max_pages}')
    print ('total ',len(recruit_items))
    for ii in range(len(recruit_items)):
        
        print ('\nstart ',ii + 20*(ipage))
        recruit = recruit_items[ii].find('a', {'class':'title dev_view'})
        if str(type(recruit)) == "<class 'NoneType'>": continue
        html_recruit, url = get_recruit_html(site, recruit)
        iframe = html_recruit.find('iframe', {'title':'상세요강'}) # html 내의 document (iframe) 처리
        html_iframe = get_iframe_html(site, iframe)

        # title, job, company, url
        title = remove_tags(html_recruit.find('title'))
        company = remove_tags(html_recruit.find('span', {'class':'coName'}))
        html_str = str(html_iframe)

        # keywords and location
        keywords = ''
        loc = ''
        assoc_keywords = html_recruit.find('article', {'id':'artKeywordSearch'})
        for key in assoc_keywords.find_all('li'):
            if str(key).find('btnLocal') > -1:
                loc += remove_tags(key).replace('&gt;', '>')
                continue
            keywords += remove_tags(key) + ', '

        # rec type
        rec = ''
        apply_qual = remove_tags(html_recruit.find('dl', {'class':'tbList'})).replace('\n','')
        apply_qual1 = apply_qual.find('경력')
        apply_qual2 = apply_qual.find('학력')
        if apply_qual2 >= 0:
            rec = apply_qual[apply_qual1+2:apply_qual2].replace(' ','')
        else:
            rec = apply_qual[apply_qual1+2:].replace(' ','')

        # 상세 채용 정보 내용
        duties = ''
        skills = ''
        preference = ''
        welfare = ''

        content = html_iframe.find('div', {'class':'detailed-summary-contents'})
        img = html_iframe.find('div', {'class':'detailReadImg'})
        if str(content) != 'None':
            ps = content.find_all('p')
            for ip in ps:
                str_ip = remove_tags(ip)
                if '담당업무' in str(ip): # duties
                    case = 1
                elif any([needs in str(ip) for needs in ['필수', '자격']]): # skills
                    case = 2
                elif '우대' in str(ip): # preference
                    case = 3
                elif any([i in str(ip) for i in ['복지', '복리']]): # welfare
                    case = 4
                elif any([i in str(ip) for i in ['전형절차']]):
                    case = -1
                try:
                    if case == 1: # duties
                        duties += str_ip + '\n'
                    elif case == 2: # skills
                        skills += str_ip + '\n'
                    elif case == 3: # preference
                        preference += str_ip + '\n'
                    elif case == 4: # welfare
                        welfare += str_ip + '\n'
                    elif case == -1: # others
                        print ('other : ',str_ip)
                except NameError:
                    pass
        elif str(img) != 'None':
            pass

        # Data에 저장
        data = [title, job, company, duties, skills, preference, welfare, loc, rec, url, keywords]
        save_1row(data)
        
    ipage += 1
print ('finished')



job 데이터 엔지니어
keyword 데이터 엔지니어
ijob, ikeyword 0, 0
max page 10

page 3 / 5
total  20

start  60
other :  전형절차
other :  ㆍ서류전형 &gt; 1차면접 &gt; 2차면접 &gt; 임원면접 &gt; 최종합격
other :  ㆍ면접일정은 추후 통보됩니다.
other :  
other :  유의사항
other :  ㆍ허위사실이 발견될 경우 채용이 취소될 수 있습니다.
other :  

start  61
other :  전형절차
other :  ㆍ서류전형 &gt; 1차면접 &gt; 2차면접 &gt; 임원면접 &gt; 최종합격
other :  ㆍ면접일정은 추후 통보됩니다.
other :  
other :  유의사항
other :  ㆍ허위사실이 발견될 경우 채용이 취소될 수 있습니다.
other :  

start  62
other :  전형절차
other :  ㆍ서류전형 &gt; 1차면접 &gt; 2차면접 &gt; 임원면접 &gt; 최종합격
other :  ㆍ면접일정은 추후 통보됩니다.
other :  
other :  유의사항
other :  ㆍ허위사실이 발견될 경우 채용이 취소될 수 있습니다.
other :  

start  63

start  64

start  65

start  66

start  67

start  68

start  69

start  70

start  71
other :  전형절차
other :  ㆍ서류전형 →  코딩테스트전형 → 1차 실무진 면접전형 → 2차 경영진 면접전형 → 연봉 및 처우협의 → 최종합격
other :  ㆍ각 전형 결과 및 면접 일정은 홈페이지 및 메일을 통해 개별적으로 안내 드리고 있습니다.
other :  ㆍ서류전형 결과 발표일 : 2022년 8월 16일 화요일 이내
other :  ㆍ입사지원서 제출일의 익월 중순까지 서류전형 결과를 안내 드리고 있습니다.
other :  ㆍ공고에 게시된 서류전형 결과 발표일보다 전형결과

In [131]:
import openpyxl
Data.to_excel('../output/DataScience_0722_1.xlsx')

In [130]:
print (url)
Data

https://www.jobkorea.co.kr//Recruit/GI_Read/38771206?Oem_Code=C1&logpath=1&stext=데이터 엔지니어&listno=40


,title,job,cname,duties,skills,preference,welfare,loc,rec type,url,keyword
0,비즈테크아이 채용 - [LG그룹] BI/DW데이터 엔지니어 채용 | 잡코리아,데이터 엔지니어,㈜비즈테크아이,,,,,서울 > 마포구,경력,https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"IT분야 직무채용관, QA·테스터·검증, Windows, 리눅스, DBA·데이터베이..."
1,웅진씽크빅 채용 - [웅진씽크빅] 정규직 데이터 엔지니어 채용 | 잡코리아,데이터 엔지니어,㈜웅진씽크빅,,,,,서울 > 중구,경력(3년이상),https://www.jobkorea.co.kr//Recruit/GI_Read/38...,"IT분야 직무채용관, DBA·데이터베이스, 데이터베이스, DBA, DW, SQL, ..."
2,텐바이텐 채용 - [텐바이텐] 데이터 엔지니어 경력직 채용 | 잡코리아,데이터 엔지니어,㈜텐바이텐,,,,,서울 > 종로구,경력(1년이상),https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"IT분야 직무채용관, DBA·데이터베이스, 데이터베이스, DW, 웹프로그래머, Py..."
3,네이버제트 채용 - 데이터 엔지니어 모집 | 잡코리아,데이터 엔지니어,네이버제트,"담당업무\nㆍ제페토의 핵심 데이터 수집, 활용 정책 수립\nㆍ제페토의 정형 비정형 ...",모집분야 및 자격요건\n데이터 엔지니어\n자격요건\nㆍ관련 분야 개발 경력 3년 이...,우대사항\nㆍ대용량 / 실시간 데이터 처리 아키텍쳐 구축 경험이 있는 분\nㆍ클라우...,,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"IT분야 직무채용관, IT·솔루션·기술영업, 웹사이트구축영업, 네트워크·서버·보안,..."
4,삼성웰스토리 채용 - B2B 온라인(E커머스) 및 Data 부문 경력사원 채용 (데...,데이터 엔지니어,삼성웰스토리㈜,"담당업무\nㆍ데이터 품질 관리\n- 핵심 Data 수집 정책 수립, DB 품질 관리...",모집분야 및 자격요건\n데이터 엔지니어\n자격요건\nㆍ데이터 가공 및 분산 처리 기...,"우대사항\nㆍ컴퓨터공학, AI, SW, 산업공학 등 관련 전공\nㆍAWS SageM...",,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"IT분야 직무채용관, 외식 직무채용관, 빅데이터·AI(인공지능), Python, 데..."
5,엘지전자 채용 - [글로벌마케팅센터] 데이터 사이언스 분야 경력채용 (데이터 엔지니...,데이터 엔지니어,엘지전자㈜,담당업무\nㆍ마케팅 분석 업무를 위한 데이터 마트 설계 / 개발 및 운영\nㆍ내부 ...,모집분야 및 자격요건\n데이터 엔지니어( 0명 )\nㆍ필수 조건\n- GCP(Goo...,우대사항\nㆍ7년 이상의 빅데이터 엔지니어링 경력\n,,서울 > 영등포구,경력,https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"IT분야 직무채용관, 빅데이터·AI(인공지능), 데이터분석, 빅데이터, 하둡(Had..."
6,카카오뱅크 채용 - [빅데이터분석] 데이터 엔지니어 | 잡코리아,데이터 엔지니어,카카오뱅크,담당업무\nㆍ분석엔지니어링\n- 데이터 기반의 분석을 위한 데이터 마트 개발 및 운...,"모집분야 및 자격요건\n데이터 엔지니어( 0명 )\n필수 경험과 역량\nㆍSQL, ...",우대사항\nㆍHadoop EcoSystem 기반의 플랫폼 환경에서 데이터 파이프라인...,,경기 > 성남시 분당구,경력,https://www.jobkorea.co.kr//Recruit/GI_Read/39...,"금융·영업 직무채용관, IT분야 직무채용관, DBA·데이터베이스, SQL, 데이터베..."
7,휴머스온 채용 - [데이터엔지니어] [데이터사이언티스트] [데이터 분석가] 경력 모...,데이터 엔지니어,주식회사 휴머스온,,,,,서울 > 강남구서울 > 서울전지역,경력(2년이상),https://www.jobkorea.co.kr//Recruit/GI_Read/38...,"이달의 우수 중소기업 일자리, 강소기업 채용관, IT분야 직무채용관, 빅데이터·AI..."
8,위메이드 채용 - 데이터 엔지니어 담당자 ( 주니어 ) | 잡코리아,데이터 엔지니어,위메이드,,,,,경기 > 성남시 분당구,경력무관,https://www.jobkorea.co.kr//Recruit/GI_Read/38...,"IT분야 직무채용관, DBA·데이터베이스, 데이터베이스, DW, 빅데이터·AI(인공..."
9,위메이드 채용 - 데이터 엔지니어 팀장 | 잡코리아,데이터 엔지니어,위메이드,,,,,경기 > 성남시 분당구,경력(5년이상),https://www.jobkorea.co.kr//Recruit/GI_Read/38...,"IT분야 직무채용관, DBA·데이터베이스, 데이터베이스, DW, 빅데이터·AI(인공..."
